# Explaining the COVID-19 cases in Montreal

## Table Of Contents

* [1. Introduction](#item1)
* [2. Data Acquisition](#item2)
    * [2.1 Data Sources](#item2_1)
    * [2.2 Data Cleaning](#item2_2)
    * [2.3 Data Wrangling](#item2_3)
* [3. Methodology](#item3)
* [4. Results](#item4)
* [5. Discussion](#item5)
* [6. Conclusion](#item6)

## 1. Introduction <a class="anchor" id="item1"></a>

The coronavirus or, otherwise known as COVID-19, has disrupted societies worldwide and brought entire industries to a halt. 

In Canada, however, not all provinces have been affected equally, with Quebec being at the top of the list with the highest number of cases. Since the beginning of the pandemic, it was obvious that Montreal would probably be the hottest spot of the province but, as weeks went by, it also became apparent that some areas of the island were more affected than others.

In this project, we will analyze and, based on this historic data, cluster the boroughs of the Montreal island with the goal of identifying the areas that are more at risk of having a spike in the near future. Furthermore, we will leverage the Foursquare API to determine the availability of healthcare locations on the island and, hopefully, explain the high number of confirmed cases in the most affected boroughs.

## 2. Data Acquisition <a class="anchor" id="item2"></a>

### 2.1 Data Sources <a class="anchor" id="item2_1"></a>

Most of the data is available at the <a href="https://santemontreal.qc.ca/en/public/coronavirus-covid-19/situation-of-the-coronavirus-covid-19-in-montreal">Santé Montréal website</a>. There are multiple data sets available on their website but, for this report, we are mainly interested in the table titled **"Numbers of confirmed cases and deaths by borough or linked city"**, which can also be downloaded in CSV format.

However, location information had to be sourced from the <a href="http://donnees.ville.montreal.qc.ca/dataset/polygones-arrondissements">Ville de Montréal website</a>, and can be found by querying **"Limite administrative de l'agglomération de Montréal"**. For our report, we picked the GEOJSON format which, apart from the usual polygon coordinates, contains data properties about area, perimeter and borough type.

Finally, we are going to compliment these data with information from Foursquare's search API about venues whose categories are related to healthcare institutions, such as hospitals, emergency rooms, medical centers, etc. The complete list of possible venues is available <a href="https://developer.foursquare.com/docs/build-with-foursquare/categories/">here</a>.

### 2.2 Data Cleaning <a class="anchor" id="item2_2"></a>

Although the data on the Santé Montréal website is presented in English and French, the downloadable CSV appears to be only in French. For convenience and ease of integration, it was decided that the CSV would be simpler to clean and maintain in the code. The first cleanup required was translating the headers. Next, we removed the noise from the table body; this included removing the comma in values above 1000, removing ***"<"*** for some very small values, and ***"*"*** for values with a citation at the footer of the table. Furthermore, the value ***"n.p."*** was replaced by 0. The finalize the cleanup process, ***NAN*** values were converted to 0, as well. The justification can be found at the footer of the table on the website:

<blockquote>
* Because of the small number of reported cases in relation with the total population, the precision of the rate value isn't optimal and should therfore be interpreted with caution
<br.>
n.p. Because of the very small number or reported cases in relation with the total population, the precision of the rate value is considered too low to be published
</blockquote>

### 2.3 Data Wrangling <a class="anchor" id="item2_3"></a>

Once the cleanup was performed, we could proceed to transforming some of the data. To do any meaningful calculations, we first cast all the numerical fields to float. We can infer the pupolation of the borough by dividing the confirmed cases by the confrmed rate per 100K people. This gave us a very close approximation of the real population numbers as reported by the last Canadian census of 2016. Unfortunately, the Senneville borough did not have a value for the confirmed rate so the population could not be calculated, the actual value from the was manually entered for that particular case.

The next step was to merge this dataframe with some useful information found in the GEOJSON file from Ville de Montreal website that we intended to use for plotting the map. At this point, some borough names had to be adjusted to be able to join the datasets. Once the names matched, we merged into the former dataset the area and the one-hot encoding of the borough types (which are either Municipality or Linked city). Now, having the area and population for each borough, we could also obtain population density.

Finally, we calculated the approximate center LL (latitude/longitude) coordinates for each borough, which will later be required for in-depth exploratory analysis and querying the Foursquare API.

## 3. Methodology <a class="anchor" id="item3"></a>

An initial attempt at exploratory analysis was to plot a choropleth map of the confirmed case count by borough to get a feel of the most affected area of the island. Clearly, the northeast was the hardest hit, along with Cote-des-Neiges/Notre-Dame-de-Grace. By distinguishing municipalities from linked cities, also gave us an insight that seemed to indicated the latter are somehow better at managing the situation.

<img src="cases.png">

We then contrasted this data with population density and the results seemed to suggest that, on average, the less densly populated boroughs are indeed the ones with less recorded cases. This also seems to coincide with the independent suburbs, although it is not a conclusive metric.

<img src="density.png">

The next piece of information that would be required is contrasting the cases to the availability of healthcare institutions on or around each borough. The Foursquare search API is queried with the LL coordinates of each borough and a radius large enough to account for the different borough shapes and forms. We further specify the venue categories that we are interested in to limit our search, as follows:

* Medical Center: **4bf58dd8d48988d104941735**
* Emergency Room: **4bf58dd8d48988d194941735**
* Hospital: **4bf58dd8d48988d196941735**
* Hospital Ward: **58daa1558bbb0b01f18ec1f7**
* Urgent Care Center: **56aa371be4b08b9a8d573526**

We generate a choropleth of the number of healthcare centers and their locations on the map so as to evaluate their distribution and availability across boroughs. At first look, it is not obvious if there are enough of each, especially, in those boroughs that are most in need. 

<img src="clinics.png">

But now that we have the exact numbers, we can calculate the number of patients per clinic, per borough. This metric will provide insight into how overloaded healthcare institutions could become during spikes of new cases. Clearly, plotting this patient-to-clinic density metric seems to indicate that the boroughs that are hardest hit are also those with the high patient density per clinic.

<img src="patients.png">

Next, we will use K-means clustering to understand how the data can be segregated in an attempt to identify the next borough where outbreaks could saturate the healthcare system of the area. For this, we pick 3 clusters (in an attempt to segregate into safe-zones, hot spots and possible candidates for future outbreaks). The following features are used from the dataset and all values are normalized:

* ConfirmedCount
* DeathCount
* BoroughType
* Density
* Emergency Room
* Hospital
* Medical Center
* PatientsPerClinic

## 4. Results <a class="anchor" id="item4"></a>

The resulting clustering are quite as expected, and can be interpreted as follows: potential candidates for future outbreak, hot-spots, and safe-zones.

#### Cluser 1:
Boroughs to watch out for. These boroughs seem to be under control but exhibit a moderate-to-relatively-high patient per clinic ratio. This could become a bottleneck for testing and patient care if cases start going up all of the sudden. The presence of boroughs with low cases like Mont-Royal, Outremont and Westmount could be explained by their location being surrounded by boroughs with higher confirmed/deatch counts, which makes them be at risk.

#### Cluster 2
Very affected boroughs. High confirmed cases, likely due to population density and high patient per clinic ratio. These are the boroughs that with the highest number of confirmed cases as seen in the first plot, namely, the northeast and Cote-des-Neiges/Notre-Dame-de-Grace.

#### Cluster 3: 
Safe zones, low confirmed cases and deaths. Usually, high ratio of number of patients to  clinic availability. These are for the most part independent suburs and the west island, they also report, on average, a lower-than-average population density.

<img src="clusters.png">

## 5. Discussion <a class="anchor" id="item5"></a>

From the clustering results, it seems apparent that the type of clinic has little effect on the number of cases, but the overall availability of them does have a direct correlation. As expected, confirmed infections and mortality are quite closely related, which makes sense as no efficient treatment is available as of yet. This leads one to believe that the role of clinics is mainly to perform testing and relieve those patients who exhibit the worst symptoms, usually requiring intensive care.

## 6. Conclusion <a class="anchor" id="item6"></a>

Although this reporty has provided a good insight into the current situation, a more thorough analysis of demographics would be required to better understand the factor that lead some boroughs to have so many more cases. Unfortunately, the data was not readily available. The closest dataset that provide this information is the Canada census 2016 but it is based on FSAs (Forward Sortation Areas) as opposed to borough names, and would require a lot of approximation to adapt it. Another option is the Ville de Montreal website that provides publications about demographiques by borough, but they are in PDF format and with no speficic structure, so they would require a lot of manual interaction to extract, which is outside of the scope of this analysis.